In [19]:
# 라이브러리 import
import numpy as np
import pandas as pd
import mariadb
import sys
from sklearn.feature_extraction.text import CountVectorizer  #텍스트(String) 벡터화
from sklearn.metrics.pairwise import cosine_similarity  #코사인 유사도


# DB Contents 데이터 불러오기

In [20]:
conn = mariadb.connect(
    user="eoeo0326",
    password="2017150009",
    host="oosoords.cqo7bp08tx6h.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    db="oosooDB"
)

contents_query = "SELECT * FROM contents"

contents_df = pd.read_sql(contents_query, conn)

contents_df.sort_values(by='vote_average', ascending=False).head(30)

,id,_type,title,genre,production_countries,vote_count,vote_average,number_of_seasons,number_of_episodes,release_date,adult,poster_path,runtime,overview,now_status,flatrate,rent,buy
24100,m_722083,movie,ಫ್ರೆಂಚ್ ಬಿರಿಯಾನಿ,"코미디,","India,",1,10.0,NaN,NaN,2020-07-24,0.0,/ixS8BiwuZrkam8l5knSL4ROXtw.jpg,116.0,,None,"Amazon Prime Video,",,
28578,t_109000,tv,이지의 코알라 월드,"가족, 다큐멘터리,","Australia,",5,10.0,2.0,16.0,2020-09-15,NaN,/6rsZXpoLkZ64nfXJ2py6ZksRUwc.jpg,NaN,"코알라 천국, 호주 마그네틱섬. 하지만 여기서도 코알라들이 어려움에 처하곤 한다. ...",Returning Series,"Netflix,",,
34399,t_88067,tv,야공중최섬량적성,"코미디,","China,",4,10.0,1.0,44.0,2019-03-25,NaN,/tQeIOuJTOfzaTv0AB5I8FBWyDh8.jpg,NaN,,Ended,"wavve, Watcha,",,
30695,t_194624,tv,のだめカンタービレ in ヨーロッパ,,,1,10.0,2.0,4.0,2008-01-04,NaN,/ktgyGvLDeRafvu9zB8fZ36pwc6T.jpg,NaN,,Returning Series,"wavve, Watcha,","Naver Store,",
17519,m_54317,movie,The Moon Inside You,"다큐멘터리,",,1,10.0,NaN,NaN,2009-03-08,0.0,/6J2jkrVzT3P4r3qKLX8drxloecM.jpg,75.0,,None,"DocAlliance Films,","DocAlliance Films,","DocAlliance Films,"
24157,m_724457,movie,Hacked,"공포,","United States of America,",1,10.0,NaN,NaN,2020-10-23,0.0,/frulCclclva18JkckRfOhstIR93.jpg,71.0,,None,,,
24158,m_724536,movie,Muhammad Ali: The Greatest,"다큐멘터리,",,1,10.0,NaN,NaN,2014-01-01,0.0,/pAb5DR5vmQ68fHHoUt9HKsBX7gn.jpg,65.0,,None,,,
17477,m_539302,movie,The Woodsman,"음악, 드라마, 판타지,",,1,10.0,NaN,NaN,2016-09-06,0.0,/5PSD8numujiggPHIaKFzOxcMUkQ.jpg,72.0,,None,"BroadwayHD,",,
24184,m_725487,movie,Mergui,"다큐멘터리,","Germany, Myanmar,",1,10.0,NaN,NaN,2019-12-01,0.0,/pzwId3z1nqZyzcIzm7iREmqprOt.jpg,42.0,,None,,,
33918,t_82936,tv,Araponga,"Soap, 드라마,","Afghanistan, Brazil,",1,10.0,1.0,143.0,1990-10-15,NaN,/gUXWKL3Kq1S7Dham046NyKqbiI8.jpg,NaN,,Ended,"wavve,",,


In [21]:
# vote_count가 적은 작품들이 vote_average 상위에 노출되는것을 알 수 있음
# 이를 해결하기 위해 IMDB의 weighted_average 사용

# IMDB Weighted Vote Average

In [22]:
# IMDB의 가중 평점 계산법
# W = (vR + mC) / (v + m)

# W: 가중 평점
# v: 해당 영화의 총 투표수
# m: 최소한의 투표 수
# R: 해당 영화의 평균 평점
# C: 모든 영화의 평균 평점

percentile = 0.9  # vote_count 상위 몇 % 를 기준으로 삼을 것인지, 0.9 = 상위 10%
m = contents_df['vote_count'].quantile(percentile)
C = contents_df['vote_average'].mean()

def weighted_vote_average(contents):
    v = contents['vote_count']
    R = contents['vote_average']
    
    return ((v * R) + (m * C)) / (v + m)

contents_df['weighted_average'] = contents_df.apply(weighted_vote_average, axis=1)

contents_df.sort_values(by='weighted_average', ascending=False).head(30)

,id,_type,title,genre,production_countries,vote_count,vote_average,number_of_seasons,number_of_episodes,release_date,adult,poster_path,runtime,overview,now_status,flatrate,rent,buy,weighted_average
6594,m_278,movie,쇼생크 탈출,"드라마, 범죄,","United States of America,",20723,8.7,NaN,NaN,1994-09-23,0.0,/oAt6OtpwYCdJI76AVtVKW1eorYx.jpg,142.0,촉망받는 은행 간부 앤디 듀프레인은 아내와 그녀의 정부를 살해했다는 누명을 쓴다. ...,None,"Netflix, wavve, Watcha,","wavve, Naver Store,","wavve, Naver Store,",8.573413
5077,m_238,movie,대부,"드라마, 범죄,","United States of America,",15465,8.7,NaN,NaN,1972-03-14,0.0,/cOwVs8eYA4G9ZQs7hIRSoiZr46Q.jpg,175.0,"시실리에서 이민온 뒤, 정치권까지 영향력을 미치는 거물로 자리잡은 돈 꼴레오네는 갖...",None,,"wavve,","wavve,",8.532297
30247,t_1396,tv,브레이킹 배드,"드라마,","United States of America,",8262,8.8,5.0,62.0,2008-01-20,NaN,/ztkUQFLlC19CCMYHW9o1zWhJRNq.jpg,NaN,2008년 1월 AMC에서 방영을 시작한 범죄 스릴러. Breaking Bad는 막...,Ended,"Netflix,",,,8.489807
2463,m_155,movie,다크 나이트,"드라마, 액션, 범죄, 스릴러,","United Kingdom, United States of America,",26725,8.5,NaN,NaN,2008-07-14,0.0,/f6dNinWX8rBM79JXKcShkfSh2oA.jpg,152.0,범죄와 부정부패를 제거하여 고담시를 지키려는 배트맨. 그는 짐 고든 형사와 패기 넘...,None,"wavve,","wavve, Naver Store,","wavve, Naver Store,",8.406362
32590,t_69050,tv,리버데일,"범죄, 드라마, 미스터리,","United States of America,",12497,8.6,6.0,102.0,2017-01-26,NaN,/6zBWSuYW3Ps1nTfeMS8siS4KUaA.jpg,NaN,"평화로운 소도시 리버데일에서 살아가는 아치와 친구들이 복잡한 연애, 섹스, 학교생활...",Returning Series,"Netflix,",,,8.400115
12147,m_424,movie,쉰들러 리스트,"드라마, 역사, 전쟁,","United States of America,",12366,8.6,NaN,NaN,1993-11-30,0.0,/oyyUcGwLX7LTFS1pQbLrQpyzIyt.jpg,195.0,2차 세계대전 당시 독일군이 점령한 폴란드. 시류에 맞춰 자신의 성공을 추구하는 기...,None,"Netflix, wavve, Watcha,","wavve, Naver Store,","wavve, Naver Store,",8.398114
31729,t_60625,tv,릭 앤 모티,"애니메이션, 코미디, Sci-Fi & Fantasy, Action & Adventu...","United States of America,",6170,8.8,5.0,51.0,2013-12-02,NaN,/8kOWDBK6XlPUzckuHDo3wwVRFwt.jpg,NaN,술에 절어 사는 천재 과학자 릭은 투정이 심한 10대 손자 모티를 데리고 외계와 대...,Returning Series,"Netflix,",,,8.395665
23041,m_680,movie,펄프 픽션,"스릴러, 범죄,","United States of America,",22599,8.5,NaN,NaN,1994-09-10,0.0,/6lXRHGoEbnnBUKsuqpL9JxD4DzT.jpg,154.0,펌프킨와 허니 버니가 레스토랑에서 강도 행각을 벌이기 시작한다. 빈센트와 그 동료 ...,None,"wavve,",,,8.389796
1478,m_13,movie,포레스트 검프,"코미디, 드라마, 로맨스,","United States of America,",22068,8.5,NaN,NaN,1994-07-06,0.0,/xdJxoq0dtkchOkUz5UVKuxn7a2V.jpg,142.0,"불편한 다리, 남들보다 조금 떨어지는 지능을 가진 포레스트 검프는 헌신적인 어머니의...",None,"Netflix, wavve, Watcha,","wavve, Naver Store,","wavve, Naver Store,",8.387228
33143,t_75006,tv,엄브렐러 아카데미,"Action & Adventure, Sci-Fi & Fantasy, 드라마,","Canada, United States of America,",7676,8.7,3.0,30.0,2019-02-15,NaN,/UBhjjSojL8z2ThtitnyWaKM8nq.jpg,NaN,초능력을 가진 남매들이 아버지의 장례식에서 다시 모인다. 그들 앞에 드러나는 가족의...,Returning Series,"Netflix,",,,8.376773


# 장르 유사도 측정

In [26]:
# CountVectorizer
count_vector = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_matrix = count_vector.fit_transform(contents_df['genre'])

genre_matrix

<35396x472 sparse matrix of type '<class 'numpy.int64'>'
	with 97574 stored elements in Compressed Sparse Row format>

In [27]:
# Cosine Similarity
genre_similarity = cosine_similarity(genre_matrix, genre_matrix)

genre_similarity[0]

array([1.       , 0.       , 0.1490712, ..., 0.       , 0.       ,
       0.       ])

In [28]:
# 입력한 영화와 장르가 유사한 영화를 반환하는 함수
def find_similar_genre(contents, similarity, contents_id, top_n=10):
    
    # 입력한 id를 가진 영화의 index 구하기 (similarity 행렬의 몇번째 인덱스인지)
    id_content = contents[contents['id'] == contents_id]
    content_index = id_content.index.values
    
    # 입력한 영화의 장르 유사도를 contents 데이터 프레임에 추가
    contents["similarity"] = similarity[content_index, :].reshape(-1,1)
    
    # 장르 유사도와 가중 평점을 기준으로 내림차순 정렬 후 상위 index 추출
    sorted_contents = contents.sort_values(by=['similarity', 'weighted_average'], ascending=False)
    sorted_contents = sorted_contents[sorted_contents.index.values != content_index]
    result = sorted_contents.index.values[ : top_n]
    
    return contents.iloc[result]

In [29]:
# 영화 '대부' 로 테스트
similar_genre_contents = find_similar_genre(contents_df, genre_similarity, 'm_238', top_n=20)
similar_genre_contents[['id', 'title', 'genre', 'weighted_average', 'similarity']]

,id,title,genre,weighted_average,similarity
6594,m_278,쇼생크 탈출,"드라마, 범죄,",8.573413,1.0
5129,m_240,대부 2,"드라마, 범죄,",8.336857,1.0
25142,m_769,좋은 친구들,"드라마, 범죄,",8.255500,1.0
31724,t_60574,피키 블라인더스,"드라마, 범죄,",8.187434,1.0
7834,m_311,원스 어폰 어 타임 인 아메리카,"드라마, 범죄,",7.871795,1.0
21276,m_640,캐치 미 이프 유 캔,"드라마, 범죄,",7.828131,1.0
20761,m_627,트레인스포팅,"드라마, 범죄,",7.746041,1.0
15584,m_4982,아메리칸 갱스터,"드라마, 범죄,",7.211047,1.0
33913,t_82883,디 액트,"드라마, 범죄,",7.149225,1.0
32465,t_67744,마인드헌터,"드라마, 범죄,",6.986842,1.0
